# Un Agente de LangChain 1.0 que puede hablar con una Base de Datos

## ¿Qué hace este código?

Este **Agente de IA** puede hablar con una base de datos SQL y responder preguntas sobre ella. Piensa en él como un asistente inteligente que sabe cómo escribir y ejecutar consultas SQL para encontrar información por ti.

In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
from langchain_community.utilities import SQLDatabase

# Fix: Specify the tables with correct capitalization to avoid error
db = SQLDatabase.from_uri(
    "sqlite:///Chinook.db",
    include_tables=['Artist', 'Album', 'Track', 'Customer', 'Invoice']  # Use exact names!
)

from langchain.tools import tool

@tool
def sql_query(query: str) -> str:

    """Obtain information from the database using SQL queries"""

    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"

from langchain.agents import create_agent

agent = create_agent(
    model="gpt-4o-mini",
    tools=[sql_query]
)

from langchain.messages import HumanMessage

question = HumanMessage(content="How many artists are in the database?")

response = agent.invoke(
    {"messages": [question]}
)

print(response["messages"][-1].content)

In [ ]:
from pprint import pprint

pprint(response['messages'])

## Expliquemos el código anterior en términos simples

#### 1. Cargando Variables de Entorno
```python
from dotenv import load_dotenv
load_dotenv()
```
- **Qué hace**: Carga las claves secretas (como tu clave API de OpenAI) desde un archivo `.env`
- **Por qué**: Mantiene tus claves API seguras y separadas de tu código
- **Consejo para principiantes**: El archivo `.env` contiene `OPENAI_API_KEY=tu-clave-aquí`

---

#### 2. Conectando a la Base de Datos
```python
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri(
    "sqlite:///Chinook.db",
    include_tables=['Artist', 'Album', 'Track', 'Customer', 'Invoice']
)
```
- **Qué hace**: Crea una conexión a un archivo de base de datos SQLite llamado `Chinook.db`
- **`include_tables`**: Limita el acceso solo a estas 5 tablas específicas por seguridad
- **Importante**: ¡Los nombres de las tablas deben coincidir EXACTAMENTE (las mayúsculas importan)!
- **Consejo para principiantes**: Esto es como abrir una puerta a la base de datos para poder consultarla

---

#### 3. Creando una Herramienta para el Agente
```python
from langchain.tools import tool

@tool
def sql_query(query: str) -> str:
    """Obtain information from the database using SQL queries"""
    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"
```
- **Decorador `@tool`**: Este marcador especial le dice a LangChain "esta función es una herramienta que el agente puede usar"
- **El docstring** (`"""Obtain information...""")`): ¡La IA lee esto para entender qué hace la herramienta!
- **Propósito de la función**: Toma una cadena de consulta SQL, la ejecuta en la base de datos y devuelve los resultados
- **Manejo de errores**: Si algo sale mal (como una consulta SQL incorrecta), captura el error y devuelve un mensaje útil
- **Consejo para principiantes**: Las herramientas son como dar habilidades especiales a la IA—en este caso, la capacidad de consultar bases de datos

---

#### 4. Creando el Agente
```python
from langchain.agents import create_agent

agent = create_agent(
    model="gpt-4o-mini",
    tools=[sql_query]
)
```
- **`create_agent`**: Función lista para producción de LangChain 1.0 para crear agentes de IA
- **`model="gpt-4o-mini"`**: Especifica qué modelo de IA usar (también puedes pasar una instancia del modelo para más control)
- **`tools=[sql_query]`**: Da al agente acceso a nuestra herramienta de consulta SQL
- **Bajo el capó**: Crea un agente basado en grafos usando LangGraph que sigue el **patrón ReAct** (Razonamiento + Actuación)
- **Consejo para principiantes**: El agente ahora puede "pensar" sobre tu pregunta y decidir cuándo usar la herramienta SQL

---

#### 5. Haciendo una Pregunta
```python
from langchain.messages import HumanMessage

question = HumanMessage(content="How many artists are in the database?")

response = agent.invoke(
    {"messages": [question]}
)
```
- **`HumanMessage`**: Un formato de mensaje especial que representa lo que tú (el humano) estás preguntando
- **`agent.invoke()`**: Ejecuta el agente con tu pregunta
- **Qué sucede internamente**:
  1. El agente lee tu pregunta
  2. Piensa: "Necesito consultar la base de datos para responder esto"
  3. Decide usar la herramienta `sql_query`
  4. Genera una consulta SQL (ej., `SELECT COUNT(*) FROM Artist`)
  5. Obtiene el resultado
  6. Formula una respuesta en lenguaje natural

---

#### 6. Obteniendo la Respuesta
```python
print(response["messages"][-1].content)
```
- **`response["messages"]`**: Contiene toda la conversación (tu pregunta + pensamientos del agente + respuesta final)
- **`[-1]`**: Obtiene el último mensaje (la respuesta final)
- **`.content`**: Extrae solo el texto de ese mensaje
- **Salida**: "There are a total of 275 artists in the database."

---

#### 7. Usando pprint para tener una vista más amplia del objeto de respuesta

```python
pprint(response["messages"])
```

Usar `pprint` nos da una vista completa del objeto de respuesta. Esta información completa es muy útil cuando queremos entender qué está sucediendo con todo detalle.

---

#### Cómo Funciona el Bucle de Razonamiento del Agente

Cuando ejecutas el agente, esto es lo que sucede paso a paso:

1. **El agente recibe la pregunta**: "How many artists are in the database?"

2. **El agente piensa** (patrón ReAct - Razonamiento):
   - "Necesito datos de una base de datos"
   - "Tengo una herramienta sql_query que puede ayudar"
   - "Debería usar una consulta COUNT en la tabla Artist"

3. **El agente actúa** (patrón ReAct - Actuación):
   - Llama: `sql_query("SELECT COUNT(*) as artist_count FROM artists;")`
   - *Obtiene error: no existe la tabla "artists"*

4. **El agente observa** el error y piensa de nuevo:
   - "El nombre de la tabla está mal, déjame probar 'artist' (singular)"

5. **El agente actúa de nuevo**:
   - Llama: `sql_query("SELECT COUNT(*) as artist_count FROM artist;")`
   - *Obtiene resultado: `[(275,)]`*

6. **El agente formula la respuesta final**:
   - Convierte el resultado bruto en lenguaje natural
   - Devuelve: "There are a total of 275 artists in the database."

Este es el poder de los agentes—pueden **razonar, usar herramientas, manejar errores e intentarlo de nuevo** hasta que resuelven el problema.

## Cómo ejecutar este código desde Visual Studio Code
* Abre el Terminal.
* Asegúrate de estar en la carpeta del proyecto.
* Asegúrate de tener el entorno de poetry activado.
* Introduce y ejecuta el siguiente comando:
    * `python 019-sql-agent.py`